In [79]:
import pandas as pd 
  
# reading csv file  
data = pd.read_csv("Data.csv")

In [80]:
class DecisionTree():
    def __init__(self, x, y, n_features, f_idxs,idxs,depth=10, min_leaf=5):
        self.x, self.y, self.idxs, self.min_leaf, self.f_idxs = x, y, idxs, min_leaf, f_idxs
        self.depth = depth
        self.n_features = n_features
        self.n, self.c = len(idxs), x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        
    def find_varsplit(self):
        #Will make it recursive later
        for i in self.f_idxs: self.find_better_split(i)
       

    def find_better_split(self, var_idx):
        #Lets write it later
        pass
    

        for i in range(0,self.n-self.min_leaf-1):
            xi,yi = sort_x[i],sort_y[i]
            lhs_cnt += 1; rhs_cnt -= 1
            lhs_sum += yi; rhs_sum -= yi
            lhs_sum2 += yi**2; rhs_sum2 -= yi**2
            if i<self.min_leaf or xi==sort_x[i+1]:
                continue

            lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
            rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
            curr_score = lhs_std*lhs_cnt + rhs_std*rhs_cnt
            if curr_score<self.score: 
                self.var_idx,self.score,self.split = var_idx,curr_score,xi

    @property
    def split_name(self): return self.x.columns[self.var_idx]
    
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]

    @property
    def is_leaf(self): return self.score == float('inf') or self.depth <= 0 
    

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx]<=self.split else self.rhs
        return t.predict_row(xi)
from sklearn import metrics

In [81]:
class RandomForest():
    def __init__(self, x, y, n_trees, n_features, sample_sz, depth=10, min_leaf=5):
        np.random.seed(12)
        if n_features == 'sqrt':
            self.n_features = int(np.sqrt(x.shape[1]))
        elif n_features == 'log2':
            self.n_features = int(np.log2(x.shape[1]))
        else:
            self.n_features = n_features
        print(self.n_features, "sha: ",x.shape[1])    
        self.x, self.y, self.sample_sz, self.depth, self.min_leaf  = x, y, sample_sz, depth, min_leaf
        self.trees = [self.create_tree() for i in range(n_trees)]

    def create_tree(self):
        idxs = np.random.permutation(len(self.y))[:self.sample_sz]
        f_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        return DecisionTree(self.x.iloc[idxs], self.y[idxs], self.n_features, f_idxs,
                    idxs=np.array(range(self.sample_sz)),depth = self.depth, min_leaf=self.min_leaf)
        
    def predict(self, x):
        return np.mean([t.predict(x) for t in self.trees], axis=0)

def std_agg(cnt, s1, s2): return math.sqrt((s2/cnt) - (s1/cnt)**2)
from sklearn.ensemble import RandomForestClassifier

In [82]:
from sklearn.model_selection import train_test_split

X=data[['SDNN', 'RMSSD']]  # Features
y=data['Result']


print(y)


0      Heart Failure
1      Heart Failure
2      Heart Failure
3      Heart Failure
4      Heart Failure
5      Heart Failure
6      Heart Failure
7      Heart Failure
8             Normal
9      Heart Failure
10     Heart Failure
11     Heart Failure
12     Heart Failure
13     Heart Failure
14     Heart Failure
15     Heart Failure
16     Heart Failure
17            Normal
18     Heart Failure
19            Normal
20            Normal
21            Normal
22            Normal
23            Normal
24     Heart Failure
25     Heart Failure
26     Heart Failure
27     Heart Failure
28     Heart Failure
29     Heart Failure
           ...      
78            Normal
79            Normal
80            Normal
81            Normal
82            Normal
83            Normal
84            Normal
85            Normal
86            Normal
87            Normal
88            Normal
89            Normal
90            Normal
91            Normal
92            Normal
93            Normal
94           

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [84]:


clf=RandomForestClassifier(n_estimators=1000)


clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9696969696969697
